In [14]:
import os
import pandas as pd
import numpy as np
import re
from typing import Tuple

In [ ]:
networks_dir = "/groups/itay_mayrose/halabikeren/plant_pollinator_networks/networks/web_of_life/"
exotic_species_path = "../raw_CSVs/exotic_species.csv"

In [9]:
exotic_species = pd.read_csv(exotic_species_path)

In [23]:
def get_split(record: pd.Series) -> Tuple[str, str]:
    row = record.values[0]
    row_components = row.split(" ")
    return " ".join(row_components[:2]), " ".join(row_components[2:])
    
exotic_species[["species", "location"]] = exotic_species.apply(get_split, axis=1, result_type="expand")
exotic_species.drop("Species", inplace=True, axis=1)

In [44]:
exotic_species_to_location = exotic_species.set_index("species")["location"].to_dict()

In [39]:
def correction_location(location: str):
    correction = {"UK": "United Kingdom", "USA": "United States"}
    if location.upper() == location:
        return correction[location]
    return location

network_metadata = pd.concat([pd.read_csv(f"{networks_dir}/binary/references.csv"), 
                              pd.read_csv(f"{networks_dir}/weighted/references.csv")])
network_metadata["location"] = network_metadata['Locality of Study'].apply(lambda row: correction_location(row.split(",")[-1].strip())) 

In [42]:
network_id_to_location = network_metadata.set_index("ID")["location"].to_dict()

In [58]:
network_index_regex = re.compile("M_PL_(\d*)_*\d*.csv")
networks_paths = [f"{networks_dir}/binary/{path}" for path in os.listdir(f"{networks_dir}/binary") if path.startswith("M_PL") and int(network_index_regex.search(path).group(1)) <= 59] + \
                 [f"{networks_dir}/weighted/{path}" for path in os.listdir(f"{networks_dir}/weighted") if path.startswith("M_PL") and int(network_index_regex.search(path).group(1)) <= 59]
networks = {os.path.basename(path): pd.read_csv(path) for path in networks_paths}
for filename in networks:
    networks[filename]["is_exotic"] = networks[filename].index.map(lambda name: True if filename in network_id_to_location and 
                                                                      name in exotic_species_to_location and
                                                                      network_id_to_location[filename] == exotic_species_to_location[name]
                                                                  else False)

In [60]:
networks_with_exotic_species = [network_id for network_id in networks if np.any(networks[network_id]["is_exotic"])]
len(networks_with_exotic_species)

0